# trying out signatures

In [13]:
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256
from Crypto.Signature import DSS

def make_keypair():
    privkey = ECC.generate(curve='P-256')
    pubkey = privkey.public_key()
    return privkey, pubkey 

def make_key(hex):
    key = ECC.import_key(bytes.fromhex(hex))
    return key

def make_hex_key(key):
    binary = key.public_key().export_key(format='DER')
    return binary.hex()

def sign_something(priv_key, something):
    signer = DSS.new(priv_key, 'fips-186-3')
    h = SHA256.new(something)
    signature = signer.sign(h)
    return signature.hex()

def verify_signature(pub_key, something, signature):
    h = SHA256.new(something)
    verifier = DSS.new(pub_key, 'fips-186-3')
    try:
        verifier.verify(h, signature)
        print ("The message is authentic.")
    except ValueError:
        print ("The message is not authentic.")

priv_key, pub_key = make_keypair()
priv_key2, pub_key2 = make_keypair()

# print(pub_key)

message = b'I accept this'
signature = sign_something(priv_key, message)
bitas = bytes.fromhex(signature)
verify_signature(pub_key, message, bitas)

hexkey = make_hex_key(pub_key)

make_key(hexkey)

priv_key.public_key()

The message is authentic.


EccKey(curve='NIST P-256', point_x=3085769700740607548571937129489161199975423117044086416704746571359204352775, point_y=114820690134230468518708106174253651445705435818183061891494980877268283905880)

# trying out transactions

In [14]:
from transaction import Transaction, TxIn, TxOut

COINBASE_AMOUNT = 20

bob_addr = "bob123"
alice_addr = "alice321"
bob_privkey, bob_pubkey = make_keypair()

# f = open('privatekey.der', 'wt')
# f.write(bob_privkey.export_key(format='DER').hex())
# f.close()

# Coinbase transaction to Bob
coinbase_in = TxIn("", 0, "")
coinbase_out = TxOut(bob_addr, COINBASE_AMOUNT)
coinbase_transaction = Transaction([coinbase_in], [coinbase_out])

# Bob's gift to Alice
bob_in = TxIn(coinbase_transaction.id, 0, "")
bob_out1 = TxOut(alice_addr, 10)
bob_out2 = TxOut(bob_addr, 10)
bobs_gift_transaction = Transaction([bob_in], [bob_out1, bob_out2])

unsignedTxIns = bobs_gift_transaction.txins
for uTxIn in unsignedTxIns:
    signature = sign_something(bob_privkey, bobs_gift_transaction.id.encode())
    uTxIn.signature = signature

# Verifying that thing is signed
signa = bytes.fromhex(unsignedTxIns[0].signature)
verify_signature(bob_pubkey, bobs_gift_transaction.id.encode(), signa)

The message is authentic.


# full-node toy model

In [15]:
from blockchain import *
from transaction import *
from wallet import *
from transaction_pool import *

# maak mijn wallet aan
toufik_wallet = 'toufiks-wallet'
init_wallet(toufik_wallet)
tfk_priv_key = get_key_from_wallet(toufik_wallet)
tfk_pub_key = tfk_priv_key.public_key()
tfk_address = get_hex_from_key(tfk_pub_key)

# maak een wallet aan voor Bob
bob_wallet = 'bobs-wallet'
init_wallet(bob_wallet)
bob_priv_key = get_key_from_wallet(bob_wallet)
bob_pub_key = bob_priv_key.public_key()
bob_address = get_hex_from_key(bob_pub_key)

print("\naddress toufik: ", tfk_address, '\n')
print("address bob: ", bob_address)

wallet already exists
wallet already exists

address toufik:  3059301306072a8648ce3d020106082a8648ce3d03010703420004ae1b936f141d4b37355897e82f86afe219e0c2dce78f040954a9ace5d6cbeeb8277ea923e75f1ed234b22286d43c9134689f53a90662465b531436adf0be2484 

address bob:  3059301306072a8648ce3d020106082a8648ce3d03010703420004913c01860019da0919211e4383a0b3dd2a11e93f49f8b51f22bb484ddbce4ed630ea25b9ddd14d285de325420a3a04c5f41db23765c6621a258dd3867b4b3e3f


In [16]:
# create the genesis transaction
genesis_in = TxIn(txout_id="", txout_index=0, signature="")
address = "3059301306072a8648ce3d020106082a8648ce3d0301070342000447e45d57be90ac1ad97c5232f7922ade8e63f9931666f1fab5f1bcaac0bdc4b83bf68c72534eb33b68865b5e5465c7ba46094961b578513aed09ed22ab8c03f3"
genesis_out = TxOut(address=address, amount=50)
genesis_transaction = Transaction(txins=[genesis_in], txouts=[genesis_out])

# create the genesis block
genesis_hash = "0000607e58623c07e2634999e38638e13b76af5b450cd7a295bc60179e6db208"
genesis_block = Block(index=0, hash=genesis_hash, previous_hash="", timestamp="inthebeninging", proof=33, data=[genesis_transaction])

# maak de blockchain aan
blockchain = Blockchain(genesis_block)

# maak de transaction pool aan (voor nu even niks bijzonders)
tx_pool = []

# maak de unspent transactions list
unspent_txouts = process_transactions(blockchain.chain[0].data, [], 0)

# mine een block
coinbase = get_coinbase_transaction(tfk_address, len(blockchain.chain))
transactions = [coinbase]
block = blockchain.generate_new_block(transactions)
blockchain.chain.append(block)

# update de unspent transactions list
unspent_txouts = process_transactions(blockchain.chain[1].data, unspent_txouts, 1)

# mine een block
coinbase = get_coinbase_transaction(tfk_address, len(blockchain.chain))
transactions = [coinbase]
block = blockchain.generate_new_block(transactions)
blockchain.chain.append(block)

# update de unspent transactions list
unspent_txouts = process_transactions(blockchain.chain[2].data, unspent_txouts, 2)

# maak een transaction
my_gift_to_bob = create_transaction(bob_address, 51, tfk_priv_key, unspent_txouts, [])

if my_gift_to_bob is not None:
    # mine een block
    coinbase = get_coinbase_transaction(tfk_address, len(blockchain.chain))
    transactions = [coinbase, my_gift_to_bob]
    block = blockchain.generate_new_block(transactions)
    blockchain.chain.append(block)

    # update de unspent transactions list
    unspent_txouts = process_transactions(blockchain.chain[3].data, unspent_txouts, 3)

    print(get_balance(bob_address, unspent_txouts))
    print(get_balance(tfk_address, unspent_txouts))    

51
99
